# Import

## library

In [1]:
import os 
import operator
import wandb
import tensorflow as tf
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import numpy as np
import pandas as pd
import math
from glob import glob
from tqdm import tqdm
from pandas.tseries.offsets import DateOffset
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings(action='ignore')

import seaborn as sns 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import matplotlib.cm as cm
from IPython.display import set_matplotlib_formats

sns.set(style='white', context='notebook', palette='deep')
line_color = ['#FFBF00','#FF7F50','#DE3163','#9FE2BF','#40E0D0','#6495ED','#117A65','#2471A3','#CCCCFF','#8E44AD','#CD5C5C' ,'#F08080','#FA8072' ,'#E9967A' ,'#FFA07A']
plt.style.use('fivethirtyeight')
plt.style.use("seaborn-white")
plt.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False
#print(plt.rcParams['font.family'])
%matplotlib inline

1 Physical GPUs, 1 Logical GPUs


In [2]:
#? 통계
#import statsmodels.api as sm
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from scipy.stats import mstats
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA

#? 평가지표
import hydroeval as he
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

## function

In [3]:
# 예측과 실제 수위를 scatter plot해주는 함수 
def scatter_plot(pred,answer):
    x = pred
    y = answer

    fig, axes = plt.subplots(1, 1, figsize=(7, 7))
    rmse,nse,r2=metric(y,x)
    axes.scatter(x, y, label='data') 
    lims = [np.min([axes.get_xlim(), axes.get_ylim()]), np.max([axes.get_xlim(), axes.get_ylim()]), ]
    axes.plot(lims, lims, 'k--', alpha=0.75, zorder=0, label='parity')
    axes.set_aspect('equal')
    axes.set_xlabel('Prediction',fontsize=25)
    axes.set_ylabel('Observation',fontsize=25)
    handles, labels = axes.get_legend_handles_labels()
    txt1="(a)   Jamsu bridge RMSE %.4f"%rmse
    axes.set_title(txt1, fontsize=25,loc='left')
    axes.xaxis.set_tick_params(labelsize=20)
    axes.yaxis.set_tick_params(labelsize=20)

    return fig

In [4]:
# 파일이 존재하는지 확인하는 함수 
def check(filepath):
    csv_files = glob(os.path.join(filepath, "*.csv"))
    if len(csv_files) > 0:return 1
    else:return 0
    
# 그래프에 rmse를 표시해주는 함수 
def plot_rmse(ax, answer, preds, label):
        ax.text(1.0, 0.95, '  {:.3f}  '.format(metric(answer,preds)[0]),
                fontsize=93, ha='right', va='top', transform=ax.transAxes)
        
# rmse와 nse를 계산해주는 함수(m단위)
def metric(y_true, y_pred):
    y_true=y_true/100
    y_pred=y_pred/100
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    nse=he.evaluator(he.nse, y_pred, y_true)
    return rmse,nse,r2

In [5]:
# 선행시간, 이동평균, 윈도우에 맞게 데이터셋을 제공 
def load_dataset(leadtime,moving_average,version):
    
    # 이동평균을 적용할 feature들 
    select_features=['cd_br','hj_br','jn_br','tl_gh_br','flow','water','wl_js_br']
    # feature engineering 할 유량들.
    fe_list=['cd_br','hj_br','jn_br']
    
    x=pd.concat([train_data,test_data],axis=0)
    x.reset_index(drop=True,inplace=True)
    x=x.set_index('ymdhm')
    x.index=pd.to_datetime(x.index)
    
    # 선행시간을 적용하기 위해 타겟을미뤄줌 
    y=x['target']
    x.drop('target',axis=1,inplace=True)
    
    # feature engineer을 안함(유량정보없음)
    if(version==0):
        x.drop(['fw_cd_br','fw_hj_br','fw_jn_br'],axis=1,inplace=True)
    # feature engineering 함(유량정보있고, 팔당댐도 엔지니어링)
    if(version==1):

        # 월과 시간에 대한 feature도 추가해줌 
        x['month'],x['hour']=x.index.month,x.index.hour

        flow = PCA(n_components=1)
        flow.fit(x[['tototf_pd_dam','inf_pd_dam']])
        transformed_data = flow.transform(x[['tototf_pd_dam','inf_pd_dam']])  # 변환된 데이터
        x['flow']=transformed_data
        
        water = PCA(n_components=1)
        water.fit(x[['sfw_pd_dam','wl_pd_dam']])
        transformed_data = water.transform(x[['sfw_pd_dam','wl_pd_dam']])  # 변환된 데이터
        x['water']=transformed_data
        
        x.drop(['tototf_pd_dam','inf_pd_dam','sfw_pd_dam','wl_pd_dam','ecpc_pd_dam'],axis=1,inplace=True)
        
        fe_list=['cd_br','hj_br','jn_br']
        for i in fe_list:
            v_name = f"{i}_pca"  # 동적으로 생성할 변수명
            f_name,w_name = "fw_"+i, 'wl_'+i
            tmp=x[[f_name,w_name]]
            globals()[v_name] = PCA(n_components=1)  # 변수 생성
            globals()[v_name].fit(tmp)  # PCA 수행
            transformed_data = globals()[v_name].transform(tmp)  # 변환된 데이터
            x[i]=transformed_data
            x.drop([f_name],axis=1,inplace=True)
            if(moving_average<1): x.drop([w_name],axis=1,inplace=True)
       
    # 이동평균을 적용
    if(moving_average>1):
        for i in range(len(select_features)):
            coln=select_features[i]+str(moving_average)+'ma'
            x[coln] = x[select_features[i]].rolling(window=moving_average).mean()
            if(i<3):x.drop('wl_'+select_features[i],axis=1,inplace=True)
            
    # train과 test를 다시 분리       
    idx = x.index.get_loc('2022-06-21 00:00:00')
    
    # test를 위해 train과 test의 범위를 선행시간만큼 조정 
    x_train=x[:idx]
    x_test=x[idx:]
    y_train = y[:idx]
    
    if(version==1 and moving_average==0):
        x_train=x_train[1:]
        y_train=y_train[1:]

    # 이동평균을 적용하면 생기는 nan을 없애주기 위함 
    if(moving_average!=0):
        x_train=x_train[moving_average:]
        y_train=y_train[moving_average:]
        
    x_train.reset_index(drop=True,inplace=True)
    x_test.reset_index(drop=True,inplace=True)
    y_train.reset_index(drop=True,inplace=True)
        
    features=['wl_js_br', 'pr_jg', 'pr_dg', 'pr_sj', 'tl_gh_br', 'month', 'hour',
       'flow', 'water', 'cd_br', 'hj_br', 'jn_br']
    
    x_trains,y_trains,x_tests=[],[],[]
    for i in tqdm(range(len(x_train)-12)):
        x_trains.append(np.array(x_train.loc[i:i+11, features]).astype(float))
        y_trains.append(np.array(y_train.loc[i+11+leadtime]).astype(float))
        
    for i in tqdm(range(len(x_test)-12)):
        x_tests.append(np.array(x_test.loc[i:i+11, features]).astype(float))
        
    x_train = np.array(x_trains)
    y_train = np.array(y_trains)    
    x_test = np.array(x_tests)
    
    dataset=[x_train,x_test,y_train]
    
    return dataset

# Data Load and Preprocessing

# Refined data load

In [33]:
data=pd.read_csv("../data/new_Refined_data.csv")
train_data=pd.read_csv("../data/refined_train_data.csv")
test_data=pd.read_csv("../data/refined_test_data.csv")
answer=pd.read_csv("../data/answer.csv")

train_data['target']=train_data['wl_js_br']
test_data['target']=0
# answer

train_data.drop(['wl_hg_br','fw_hg_br','wl_gj_br','wl_pd_br','fw_pd_br','ymdhm'],axis=1,inplace=True)
test_data.drop(['wl_hg_br','fw_hg_br','wl_gj_br','wl_pd_br','fw_pd_br','ymdhm'],axis=1,inplace=True)

In [34]:
test_data=pd.concat([train_data[-6:],test_data])
test_data.reset_index(drop=True,inplace=True)
test_data

train_data=train_data[:-12]
train_data

,wl_js_br,wl_cd_br,wl_hj_br,wl_jn_br,fw_cd_br,fw_hj_br,fw_jn_br,wl_pd_dam,inf_pd_dam,sfw_pd_dam,ecpc_pd_dam,tototf_pd_dam,pr_jg,pr_dg,pr_sj,tl_gh_br,target
0,271.2,274.7,258.3,962.0,248.40,356.89,2.70,25.11,145.51,215.42,28.58,145.51,0.0,0.0,0.0,538.5,271.2
1,271.2,274.7,258.3,962.0,248.40,356.89,2.70,25.11,144.97,215.42,28.58,144.97,0.0,0.0,0.0,534.6,271.2
2,270.2,274.7,258.3,962.0,248.40,356.89,2.70,25.11,144.88,215.42,28.58,144.88,0.0,0.0,0.0,527.2,270.2
3,270.2,274.7,258.3,962.0,248.40,356.89,2.70,25.12,756.53,215.79,28.21,144.87,0.0,0.0,0.0,517.7,270.2
4,270.2,274.7,258.3,962.0,248.40,356.89,2.70,25.12,145.04,215.79,28.21,145.04,0.0,0.0,0.0,507.3,270.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13249,285.2,293.7,274.3,976.0,355.94,528.33,5.72,25.06,293.03,213.59,30.41,293.03,0.0,0.0,0.0,333.7,0.0
13250,285.2,292.7,274.3,976.0,349.79,528.33,5.72,25.06,293.37,213.59,30.41,293.37,0.0,0.0,0.0,345.0,0.0
13251,285.2,291.7,274.3,976.0,343.69,528.33,5.72,25.06,293.86,213.59,30.41,293.86,0.0,0.0,0.0,356.5,0.0
13252,284.2,292.7,274.3,976.0,349.79,528.33,5.72,25.06,294.04,213.59,30.41,294.04,0.0,0.0,0.0,368.4,0.0


,wl_js_br,wl_cd_br,wl_hj_br,wl_jn_br,fw_cd_br,fw_hj_br,fw_jn_br,wl_pd_dam,inf_pd_dam,sfw_pd_dam,ecpc_pd_dam,tototf_pd_dam,pr_jg,pr_dg,pr_sj,tl_gh_br,target
0,370.2,385.7,350.3,964.0,1164.25,1821.18,2.43,24.870,463.00,221.44,22.56,561.00,0.0,0.0,0.0,284.9,370.2
1,367.2,376.7,350.3,964.0,1063.05,1821.18,2.43,24.889,230.10,222.13,21.87,131.80,0.0,0.0,0.0,270.3,367.2
2,365.2,373.7,350.3,964.0,1029.32,1821.18,2.43,24.896,333.70,222.33,21.67,137.10,0.0,0.0,0.0,257.0,365.2
3,362.2,370.7,349.3,964.0,999.00,1798.65,2.43,24.895,326.40,222.31,21.69,129.80,0.0,0.0,0.0,244.2,362.2
4,361.2,366.7,347.3,964.0,958.58,1753.59,2.43,24.892,228.10,222.18,21.82,129.80,0.0,0.0,0.0,231.5,361.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497935,273.2,275.7,259.3,962.0,253.57,366.60,2.70,25.090,149.49,214.69,29.31,149.49,0.0,0.0,0.0,426.9,273.2
497936,273.2,275.7,258.3,962.0,253.57,356.89,2.70,25.100,757.75,215.05,28.95,147.75,0.0,0.0,0.0,443.1,273.2
497937,273.2,276.7,258.3,962.0,258.79,356.89,2.70,25.090,0.00,214.69,29.31,146.96,0.0,0.0,0.0,459.1,273.2
497938,273.2,277.7,258.3,962.0,264.07,356.89,2.70,25.090,146.32,214.69,29.31,146.32,0.0,0.0,0.0,475.1,273.2


In [35]:
# Convert 'ymdhm' column to datetime format
train_data['ymdhm'] = pd.to_datetime(train_data['ymdhm'])
# Set 'ymdhm' as the index
train_data.set_index('ymdhm', inplace=True)

test_data['ymdhm'] = pd.to_datetime(test_data['ymdhm'])
# Set 'ymdhm' as the index
test_data.set_index('ymdhm', inplace=True)


KeyError: 'ymdhm'

In [36]:
from sklearn.preprocessing import MinMaxScaler


# Normalize the features
scaler = MinMaxScaler()
train_data = pd.DataFrame(scaler.fit_transform(train_data), columns=train_data.columns, index=train_data.index)
test_data = pd.DataFrame(scaler.transform(test_data), columns=test_data.columns, index=test_data.index)

In [37]:
# Create sequences
def create_sequences(input_data, tw):
    train_seq,train_label = [], []
    L = len(input_data)
    for i in range(L-tw):
        train_seq.append(input_data.loc[i:i+tw-1,:].drop(['target'],axis=1).values)
        train_label.append(input_data.loc[i+tw,'target'])
    return np.array(train_seq),np.array(train_label)

# Define the sequence length
sequence_length = 12

# Create sequences

# Separate inputs and targets
x_train, y_train = create_sequences(train_data, sequence_length)

# Separate inputs and targets
x_test, _ = create_sequences(test_data, sequence_length) 

x_train.shape, y_train.shape,x_test.shape


((497928, 12, 16), (497928,), (13242, 12, 16))

In [39]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D,Dropout 

from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()

input_shape = (x_train.shape[1], x_train.shape[2])

model = Sequential()
model.add(GRU(256, input_shape=input_shape))
model.add(Dense(1, activation='relu'))

optimizer = tf.optimizers.RMSprop(0.001)

model.compile(optimizer=optimizer,loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])


model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, 256)               210432    
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 210,689
Trainable params: 210,689
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(x_train, y_train, epochs=10, batch_size=512)

Train on 497928 samples
Epoch 1/10
497928/497928 [==============================] - 83s 167us/sample - loss: 9.7689e-04 - root_mean_squared_error: 0.0313
Epoch 2/10
497928/497928 [==============================] - 68s 136us/sample - loss: 6.7443e-05 - root_mean_squared_error: 0.0082
Epoch 3/10
497928/497928 [==============================] - 58s 116us/sample - loss: 4.2320e-05 - root_mean_squared_error: 0.0065
Epoch 4/10
 83456/497928 [====>.........................] - ETA: 36s - loss: 3.4438e-05 - root_mean_squared_error: 0.0059

KeyboardInterrupt: 

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
from tensorflow.keras.models import load_model

model.save('tcn_model(1000).h5')

In [ ]:
y_pred=pd.DataFrame(y_pred)

In [ ]:
fig=plt.figure(figsize=(15,10))
plt.plot(y_pred) # 선행시간 10m
plt.plot(answer) # 선행시간 30m
plt.legend(['pred','answer'],fontsize=20)
plt.show();

In [ ]:
features=['wl_js_br', 'pr_jg', 'pr_dg', 'pr_sj', 'tl_gh_br', 'month', 'hour',
       'flow', 'water', 'cd_br', 'hj_br', 'jn_br']

In [ ]:
import shap

np.bool = np.bool_
np.int = np.int_

explainer = shap.DeepExplainer(model, x_train[:150])

shap_values = explainer.shap_values(x_test[:100],check_additivity=False)

shap_valuesnp = np.array(shap_values) 
shap_valuesnp = np.reshape(shap_valuesnp,(int(shap_valuesnp.shape[1]),int(shap_valuesnp.shape[2]),int(shap_valuesnp.shape[3]))) 
shap_abs = np.absolute(shap_valuesnp) 
sum_0 = np.sum(shap_abs,axis=0) 


fig, axs = plt.subplots(3, 2, figsize=(10, 12))
axs = axs.flatten()
x_pos=[i for i in range(0,12)]
for i, ax in enumerate(axs):
    ax.barh(x_pos, sum_0[5 - i])
    ax.set_yticks(x_pos)
    ax.set_yticklabels(features)
    ax.set_title(f"time-{i+1}")
    ax.set_xlim(0, 1500)  # x축 범위 설정

plt.tight_layout()
plt.show();


#shap.summary_plot(shap_values[0], plot_type = 'bar', feature_names = features*6)


In [ ]:
average_shap_values = np.mean(shap_valuesnp, axis=0)

shap.summary_plot(average_shap_values, plot_type = 'bar', feature_names = features)

In [ ]:
shap.waterfall_plot(shap_values)
shap.plots.bar(shap_values)